In [3]:
!rm -r sample_data

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-b55addb2-7708-2c33-a4af-604e8c75875f)


In [2]:
%tensorflow_version 1.x
import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__))

TensorFlow 1.x selected.
Tensorflow version: 1.15.2


In [4]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install requirements

In [5]:
!pip install pydicom
!pip install SimpleITK
!pip install vtk

     |████████████████████████████████| 35.5MB 91kB/s 
     |████████████████████████████████| 42.5MB 72kB/s 
     |████████████████████████████████| 103.4MB 30kB/s 


In [6]:
import os
os.environ['PYTHONPATH'] += ":/content"
!echo $PYTHONPATH

/tensorflow-1.15.2/python3.6:/env/python:/content


## Import data

In [7]:
!git clone https://github.com/manigalati/ukbb_cardiac

Cloning into 'ukbb_cardiac'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 466 (delta 66), reused 47 (delta 39), pack-reused 377
Receiving objects: 100% (466/466), 398.25 KiB | 758.00 KiB/s, done.
Resolving deltas: 100% (279/279), done.


## Prepare dataset

In [16]:
%cd /content/ukbb_cardiac/

/content/ukbb_cardiac


In [ ]:
!mkdir dataset & unzip -q "/content/drive/My Drive/tesi/training.zip" -d dataset/
!mv dataset/training dataset/train 

In [ ]:
!cp -r dataset/train dataset/validation 

In [17]:
!unzip -q "/content/drive/My Drive/tesi/testing.zip" -d dataset/ 
!mv dataset/testing/testing dataset/tmp && rm -r dataset/testing && mv dataset/tmp dataset/test

## Training


In [ ]:
!python common/train_network.py

Output streaming troncato alle ultime 5000 righe.
  training loss:		0.048585
  training accuracy:		98.74%
Iteration 278: training...
  Select /content/ukbb_cardiac/dataset/train/patient100/patient100_frame01.nii.gz /content/ukbb_cardiac/dataset/train/patient100/patient100_frame01_gt.nii.gz
  Select /content/ukbb_cardiac/dataset/train/patient041/patient041_frame01.nii.gz /content/ukbb_cardiac/dataset/train/patient041/patient041_frame01_gt.nii.gz
Iteration 278 of 1000 took 0.168s
  training loss:		0.072324
  training accuracy:		97.91%
Iteration 279: training...
  Select /content/ukbb_cardiac/dataset/train/patient087/patient087_frame01.nii.gz /content/ukbb_cardiac/dataset/train/patient087/patient087_frame01_gt.nii.gz
  Select /content/ukbb_cardiac/dataset/train/patient048/patient048_frame01.nii.gz /content/ukbb_cardiac/dataset/train/patient048/patient048_frame01_gt.nii.gz
Iteration 279 of 1000 took 0.190s
  training loss:		0.071810
  training accuracy:		97.71%
Iteration 280: training...
 

## Testing

In [21]:
!cp -r "/content/drive/My Drive/tesi/20180914_Bai2018_Article_AutomatedCardiovascularMagneti_RESULTS/model" /content/ukbb_cardiac/model

In [28]:
import os
for dir in os.listdir("dataset/test"):
  if os.path.isdir("dataset/test/"+dir):
    source="dataset/test/"+dir+"/*_4d.nii.gz"
    dst="dataset/test/"+dir+"/sa.nii.gz"
    !mv $source $dst

In [29]:
!python common/deploy_network.py


2020-09-14 13:26:23.770803: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-09-14 13:26:23.786014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-09-14 13:26:23.786863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-09-14 13:26:23.787150: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-09-14 13:26:23.788820: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-09-14 13:26:23.790865: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcu

In [31]:
import os
for dir in os.listdir("dataset/test"):
  if os.path.isdir("dataset/test/"+dir):
    source="dataset/test/"+dir+"/seg_sa_ED.nii.gz"
    dst='"/content/drive/My Drive/tesi/20180914_Bai2018_Article_AutomatedCardiovascularMagneti_RESULTS/results/'+dir+'_ED.nii.gz"'
    !cp $source $dst
    source="dataset/test/"+dir+"/seg_sa_ES.nii.gz"
    dst='"/content/drive/My Drive/tesi/20180914_Bai2018_Article_AutomatedCardiovascularMagneti_RESULTS/results/'+dir+'_ES.nii.gz"'
    !cp $source $dst

In [35]:
%ls

aortic/  data/             __init__.py  model/        README.md
assoc/   dataset/          LICENSE      par/          short_axis/
common/  demo_pipeline.py  long_axis/   __pycache__/  third_party/
